### Description ###
This notebook contains useful functions for problems and tasks that appeared on HW1, including:
1. Generate the stream of future cash flows, including coupons and principal, for a single bond given its basic quote info.
2. Calculate a bond's YTM given its cash flow stream, its market price, and the current date.
3. Generate a cash flow matrix given basic quote info for a collection of bonds.
4. Clean and aggregate the cash flow matrix to allow for bootstrapping or estimating the spot curve.
5. Convert between discount rates and discount factors flexibly.
6. Apply the Nelson-Siegel model to build a spot curve.

### Imports ###

In [2]:
from datetime import datetime
from pandas.core.series import Series
import pandas as pd
import numpy as np
from typing import Dict, Union
from scipy.optimize import minimize

In [165]:
# load quote data for testing functions

quote_df = pd.read_excel('../Data/treasury_quotes_2024-10-31.xlsx', sheet_name = 'selected quotes')

## Functions and Example Usage ##

In [4]:
def get_bond_cash_flows(
    quote_df: pd.DataFrame,
    bond_id: Union[int, str],
    freq: int = 2,
    fv: float = 100,
    cols: Dict[str, str] = {
        'treasury number': 'KYTREASNO',
        'quote date': 'quote date',
        'issue date': 'issue date',
        'maturity date': 'maturity date',
        'cpn rate': 'cpn rate'
    }
) -> pd.DataFrame:
    """
    Generate a series of future cash flows for a single bond given its ID and a dataframe of all the quotes.

    Parameters:
        quote_df (pd.DataFrame): DataFrame containing many bonds quote information.
        bond_id (int | str): ID of the bond for which to generate cash flows.
        freq (int): frequency of coupon payment per year (default 2: semiannual)
        fv (float): face value of bond (default 100)
        cols (Dict[str, str]): names of columns necessary (just in case they are different)

    Returns:
        pd.DataFrame: A DataFrame containing the future cash flow dates and corresponding cash flows.
    """
    # Locate the bond by ID
    bond = quote_df[quote_df[cols['treasury number']] == bond_id].iloc[0]
    
    # Extract bond details
    quote_date = pd.to_datetime(bond[cols['quote date']])
    issue_date = pd.to_datetime(bond[cols['issue date']])
    maturity_date = pd.to_datetime(bond[cols['maturity date']])
    coupon_rate = bond[cols['cpn rate']]
    
    # Initialize an empty list to store cash flow information
    cash_flows = []
    
    # Generate cash flow dates (every 6 months after the issue date)
    current_date = issue_date
    while current_date < maturity_date:
        cash_flows.append((current_date, coupon_rate / freq))
        current_date += pd.DateOffset(months=int(12/freq))


    cash_flows.append((maturity_date, fv + coupon_rate / freq))
    
    # Convert to a DataFrame
    cash_flows_df = pd.DataFrame(cash_flows, columns=['date', 'cash flow'])
    
    return cash_flows_df[cash_flows_df['date'] > quote_date]

In [172]:
cashflows = get_bond_cash_flows(quote_df,208046)
cashflows

,date,cash flow
4,2025-04-30,1.75
5,2025-10-30,1.75
6,2026-04-30,1.75
7,2026-10-30,1.75
8,2027-04-30,1.75
9,2027-10-30,1.75
10,2028-04-30,1.75
11,2028-10-30,1.75
12,2029-04-30,1.75
13,2029-10-30,1.75


In [173]:
def calculate_ytm(cash_flow_df: pd.DataFrame, current_date: str, market_price: float) -> float:
    """
    Calculate the yield to maturity (YTM) of a bond given its cash flows and current price.

    Parameters:
        cash_flow_df (pd.DataFrame): DataFrame with columns 'date' (datetime) and 'cash flow' (float) for a single bond.
        current_date (str): The current date as a string. Eg. "2025-01-26".
        market_price (float): The current market price of the bond.

    Returns:
        float: Yield to maturity (YTM) as an annualized rate (semi-annual compounding).
    """
    
    # Ensure date inputs are in datetime format
    current_date = pd.to_datetime(current_date)
    cash_flow_df['date'] = pd.to_datetime(cash_flow_df['date'])

    # Filter out dates from before the current date
    cash_flow_df = cash_flow_df[cash_flow_df['date'] >= current_date]
    
    # Calculate the time to cash flows in years (semi-annual periods)
    cash_flow_df['TTCF'] = (cash_flow_df['date'] - current_date).dt.days / 365.25 

    # Define the present value function
    def pv_function(ytm):
        pv = sum(
            row['cash flow'] / (1 + ytm / 2) ** (row['TTCF'] * 2)
            for _, row in cash_flow_df.iterrows()
        )
        return (pv - market_price) ** 2  # Return the squared error between PV and actual price

    # Initial guess for the yield 
    initial_guess = 0.04

    # Minimize the present value function to find the YTM
    result = minimize(pv_function, initial_guess, bounds=[(0, None)])

    # Return the annualized YTM
    return result.x[0] if result.success else None

In [174]:
calculate_ytm(cashflows, '2024-10-31', 96.7)

0.04181968110608598

### Buiding and Processing the Cashflow Matrix ###

In [4]:
def make_cashflow_matrix(quote_df: pd.DataFrame, bond_ids: list[str] | Series) -> pd.DataFrame:
    """
    Combine cash flow streams of multiple bonds into a matrix.

    Parameters:
        quote_df (pd.DataFrame): DataFrame containing bond information.
        bond_ids (list[str] | Series): List/series of bond IDs to include in the matrix.

    Returns:
        pd.DataFrame: A DataFrame where rows are indexed by bond IDs, 
                      columns are unique cash flow dates, and values are cash flows.
    """
    # Initialize a dictionary to store cash flow data for each bond
    all_cash_flows = {}
    
    for bond_id in bond_ids:
        # Generate cash flows for the bond
        cash_flows_df = get_bond_cash_flows(quote_df, bond_id)
        
        # Store the cash flow data with the bond ID as the key
        all_cash_flows[bond_id] = cash_flows_df.set_index('date')['cash flow']
    
    # Combine all cash flow series into a single DataFrame
    combined_cash_flows = pd.DataFrame(all_cash_flows).fillna(0).T
    
    # Sort the columns (dates) for clarity
    combined_cash_flows = combined_cash_flows.rename_axis('bond id', axis='index')
    
    return combined_cash_flows

In [190]:
C = make_cashflow_matrix(quote_df, quote_df['KYTREASNO'])
C

date,2025-04-30,2025-10-30,2025-10-31,2026-04-30,2026-10-30,2026-10-31,2027-04-30,2027-10-30,2027-10-31,2028-04-30,...,2028-10-31,2029-04-30,2029-10-30,2029-10-31,2030-04-30,2030-10-30,2030-10-31,2031-04-30,2031-10-30,2031-10-31
bond id,,,,,,,,,,,,,,,,,,,,,
208042,101.9375,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
208129,2.5000,2.5000,102.5,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
208213,2.4375,2.4375,0.0,102.4375,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
208302,2.0625,2.0625,0.0,2.0625,2.0625,102.0625,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
207875,1.3750,1.3750,0.0,1.3750,1.3750,0.0000,101.3750,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
207960,2.0625,2.0625,0.0,2.0625,2.0625,0.0000,2.0625,2.0625,102.0625,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
208044,1.7500,1.7500,0.0,1.7500,1.7500,0.0000,1.7500,1.7500,0.0000,101.7500,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
208132,2.4375,2.4375,0.0,2.4375,2.4375,0.0000,2.4375,2.4375,0.0000,2.4375,...,102.4375,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
208216,2.3125,2.3125,0.0,2.3125,2.3125,0.0000,2.3125,2.3125,0.0000,2.3125,...,0.0000,102.3125,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [189]:
# this function should be sufficient if he gives us idealized data like in HW 1.

def consolidate_by_month(cash_flow_matrix: pd.DataFrame) -> pd.DataFrame:
    """
    Consolidate cash flows within the same month from an existing cash flow matrix. Only works for idealized data (maturity in perfect intervals).

    Parameters:
        cash_flow_matrix (pd.DataFrame): A DataFrame where rows are indexed by bond IDs,
                                         columns are cash flow dates, and values are cash flows.

    Returns:
        pd.DataFrame: A DataFrame where columns represent months, and cash flows within
                      the same month are aggregated.
    """
    # Convert column names to datetime for grouping
    cash_flow_matrix.columns = pd.to_datetime(cash_flow_matrix.columns)
    
    # Group columns by month (YYYY-MM format)
    grouped_columns = cash_flow_matrix.groupby(cash_flow_matrix.columns.to_period('M'), axis=1).sum()
    
    # Rename columns back to strings (YYYY-MM format)
    grouped_columns.columns = grouped_columns.columns.astype(str)

    # Make sure last cash flow is correct (assuming FV 100 for each treasury)
    fv = 100
    for i in range(len(grouped_columns)): 
        if i == 0:
            continue
        nonzero_indices = np.flatnonzero(grouped_columns.iloc[i].to_numpy())
        coupon = grouped_columns.iloc[i][0]
        correct_value = fv + coupon
        if len(nonzero_indices) > 0:  # If there is at least one nonzero value
            last_nonzero_idx = nonzero_indices[-1]  # Get the last nonzero index
            grouped_columns.iat[i, last_nonzero_idx] = correct_value

    return grouped_columns

In [191]:
consolidate_by_month(C)

/var/folders/2d/tstl7gwn2c3g61fd1t7fk8sw0000gn/T/ipykernel_53096/2795087802.py:17: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped_columns = cash_flow_matrix.groupby(cash_flow_matrix.columns.to_period('M'), axis=1).sum()


date,2025-04,2025-10,2026-04,2026-10,2027-04,2027-10,2028-04,2028-10,2029-04,2029-10,2030-04,2030-10,2031-04,2031-10
bond id,,,,,,,,,,,,,,
208042,101.9375,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000
208129,2.5000,105.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000
208213,2.4375,2.4375,102.4375,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000
208302,2.0625,2.0625,2.0625,104.1250,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000
207875,1.3750,1.3750,1.3750,1.3750,101.3750,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000
207960,2.0625,2.0625,2.0625,2.0625,2.0625,104.1250,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000
208044,1.7500,1.7500,1.7500,1.7500,1.7500,1.7500,101.7500,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000
208132,2.4375,2.4375,2.4375,2.4375,2.4375,2.4375,2.4375,104.8750,0.0000,0.0000,0.0000,0.0000,0.0000,0.000
208216,2.3125,2.3125,2.3125,2.3125,2.3125,2.3125,2.3125,2.3125,102.3125,0.0000,0.0000,0.0000,0.0000,0.000


In [1]:
def generate_discount_factor_perfect(cash_flow_matrix: pd.DataFrame, prices: pd.DataFrame, freq: int = 2) -> pd.DataFrame:
    """
    Generates discount factors with TTM given cash flow matrix, prices (only for perfect system)

    Parameters:
        cash_flow_matrix (pd.DataFrame): A DataFrame where rows are indexed by bond IDs,
                                         columns are cash flow dates, and values are cash flows.
        prices (pd.DataFrame): dataframe of the prices of the bonds
        freq (int): how often we want to get the discount factor (2 would be semiannual)
        

    Returns:
        pd.DataFrame: A DataFrame where the discount factors and times are listed
    """

    matrix = cash_flow_matrix.values
    inverse_matrix = np.linalg.inv(matrix)
    z = inverse_matrix @ prices
    z.rename(columns={z.columns[0]: 'Spot Discount Factor'}, inplace = True)
    z.index = cash_flow_matrix.columns
    z['T - t'] = np.arange(1/freq, 1/freq * (len(z) + 1), 1/freq)

    return z

NameError: name 'pd' is not defined

In [192]:
# this function must be used if he gives us real data, with linear dependence

def remove_singular_securities(cash_flow_matrix):
    '''Given a cash flow matrix, remove the securities which pay cash flows on dates where no other security pays cash flows. Then, check for 
        dates which now have no cash flows and remove these.
        Intuition: these steps will prevent linear dependence in the cash flow matrix.'''
    
    # Step 1: Find columns where all elements are < 100
    columns_to_exclude = cash_flow_matrix.columns[(cash_flow_matrix < 100).all(axis=0)]
    
    # Step 2: Identify rows with non-zero values in these columns
    rows_to_exclude = cash_flow_matrix[columns_to_exclude].any(axis=1)
    
    # Step 3: Filter out rows with non-zero values in those columns (preserve index)
    filtered_matrix = cash_flow_matrix[~rows_to_exclude]
    
    # Step 4: Remove columns containing only zeros
    filtered_matrix = filtered_matrix.loc[:, (filtered_matrix != 0).any(axis=0)]

    return cash_flow_matrix

## Converting between Discount Rates and Factors ##

In [3]:
def convert_discount_rate_to_discount_factor(discount_rate: float, ttm: float, compounding_freq):
    ''' 
    Given an annualized discount rate, a time to maturity, and a compounding frequency, convert the discount rate into a discount factor.

    Params:
        discount_rate (float): the annualized discount rate (as a decimal).
        ttm (float): time to maturity in years.
        compounding_freq (int or str): the number of compounding periods in a year. "continuous" if continuous compounding. 
    '''
    if compounding_freq=='continuous':
        return np.exp(-discount_rate * ttm)
    else:
        return (1 + discount_rate/compounding_freq)^(-compounding_freq * ttm)

In [5]:
def convert_discount_factor_to_discount_rate(discount_factor: float, ttm: float, compounding_freq):
    ''' Given a discount factor, a time to maturity in years, and a compounding frequency, convert the discount factor to an annualized discount rate.

        Params:
            discount_factor (float): the number f such that cash flow * f = present value.
            ttm (float): time to maturity in years.
            compounding_freq (int or str): the number of compounding periods in a year. "continuous" if continuous compounding. 
        
    '''
    if compounding_freq=='continuous':
        return np.log(1/discount_factor)/ttm
    else:
        return ((1/discount_factor)**(1/compounding_freq/ttm) - 1) * compounding_freq

## Nelson Siegel ##
The following functions allow us to fit the Nelson-Siegel model. Here is a brief overview of the process: 

Think of NS as a parameterized spot curve model: we input parameters, and it spits out a yield curve. To choose the parameters, we take an optimization approach: for any set of parameters, we build the spot curve and use it to price each of our bonds through present value pricing. We then compare the predicted prices to the actual prices, and aggregate into a loss function. Finally, we use gradient descent (implicitly) to select the parameters which minimize the loss. This optimal set of parameters specifies the NS model.

In [6]:
def nelson_siegel(theta, T):
    """
    Applies the Nelson-Siegel model to predict the spot rate for a given maturity. 

    Parameters:
    theta (iterable): a list of parameters [theta_0, theta_1, theta_2, lambda].
    T (float): time to maturity.

    """
    theta_0, theta_1, theta_2, lambd = theta

    # Apply the Nelson-Siegel formula
    term1 = theta_0
    term2 = (theta_1 + theta_2) * (1 - np.exp(-T / lambd)) / (T / lambd)
    term3 = -theta_2 * np.exp(-T / lambd)
    
    r = term1 + term2 + term3
    return r

In [7]:
def years_between(quote_date, target_date):
    """
    Compute the number of years between a quote date and a target date.
    """
    # Convert both dates to pandas datetime
    quote_date = pd.to_datetime(quote_date, format='%Y-%m')
    target_date = pd.to_datetime(target_date, format='%Y-%m')
    
    # Compute the time difference in years
    years_between = (target_date - quote_date).days / 365.0
    return years_between

In [8]:
def calculate_discount_factors(quote_date, cash_flow_dates, theta):
    """
    Calculate discount factors for each cash flow date using the Nelson-Siegel model.

    Parameters:
    quote_date : str
        The quote date in the format 'YYYY-MM-DD'.
    cash_flow_dates : list of str
        List of cash flow dates in the format 'YYYY-MM-DD'.
    theta : list or array-like
        A vector of parameters [theta_0, theta_1, theta_2, lambda].

    Returns:
    numpy.ndarray
        A vector of discount factors, one for each cash flow date.
    """

    # Calculate discount factors
    discount_factors = []
    for cash_flow_date in cash_flow_dates:
        ttm = years_between(quote_date, cash_flow_date)  # Time to maturity
        if ttm <= 0:
            discount_factors.append(1.0)  # If ttm is zero or negative, no discounting
            continue
        r = nelson_siegel(theta, ttm)  # Get discount rate
        discount_factor = np.exp(-r * ttm)  # Convert rate to discount factor
        discount_factors.append(discount_factor)
    
    return np.array(discount_factors)

In [9]:
def compute_loss(cash_flow_matrix, theta, quote_date, market_prices):
    ''' 
    Calculate the loss for a given set of NS parameters theta.
    
    Inputs: 
    cash_flow_matrix: processed matrix of cash flows
    theta: vector of NS parameters
    market_prices: the market prices of all securities in the cash flow matrix. 
    '''
    # Get a series of cash flow dates.
    cash_flow_dates = cash_flow_matrix.columns
    
    # Compute the discount factors by applying NS. 
    discount_factors = calculate_discount_factors(quote_date, cash_flow_dates, theta)

    # Predict the bond prices based on the discount factors.
    price_predictions = cash_flow_matrix @ discount_factors

    # Compute the mean squared error. This is what we want to minimize.
    mse = np.mean((market_prices - price_predictions) ** 2)
    return mse

In [11]:
from scipy.optimize import minimize

def find_optimal_theta(cash_flow_matrix, quote_date, market_prices, initial_theta):
    """
    Finds the optimal theta that minimizes the loss function.
    
    Parameters:
    cash_flow_matrix : pd.DataFrame
        The cash flow matrix where columns represent cash flow dates and rows represent securities.
    quote_date : str
        The quote date in the format 'YYYY-MM-DD'.
    market_prices : pd.Series
        The market prices of the securities.
    initial_theta : list or array-like
        The initial guess for the parameters theta.

    Returns:
    result : OptimizeResult
        The result of the optimization containing the optimal theta.
    """
    # Define the objective function (loss function)
    def objective(theta):
        return compute_loss(cash_flow_matrix, theta, quote_date, market_prices)
    
    # Perform the minimization using 'BFGS' (or another method if preferred)
    result = minimize(objective, initial_theta, method='BFGS')
    
    return result

## PCA ##